[This notebook describes the behavior in Julia 0.5 and later. There were some significant differences in earlier versions of Julia that are not described here.]

##  Defining new types

Types form one of the cornerstones of Julia, enabling both its elegance and speed (despite the fact that one usually explicitly refers to types much less than in, say, C++).

In this notebook, we will look at how to define new types in Julia. This is very common when you need a certain **kind (type) of behaviour** that is not captured by the "built-in" types.

User-defined types are on the same footing as the "built-in" types in the standard library (`Int`, `Float64`, etc.), which are themselves defined in Julia's base library (which is written in Julia). The compiler is often able to "optimize away" the type structure, so that the resulting code is just as efficient as it would have been if you had not used types and had just written out the corresponding operations. We are thus at liberty to use types to structure our code and reflect the model of the world that we wish to implement, without suffering from poor performance.

## Basic type definitions

We will look at a concrete example that arose naturally in real code in the author's pacakage 
[`IntervalConstraintProgramming.jl`](https://github.com/dpsanders/IntervalConstraintProgramming.jl): defining a type  to represent the volume of an object. This is similar to a [post on the `julia-users` mailing list](https://groups.google.com/forum/#!searchin/julia-users/probability/julia-users/PpXLxHajsfA/UWpmJZd2BQAJ) about defining a type to represent a probability.  

Despite the fact that the type contains only a single field, many of the confusing points about types in Julia can be best appreciated in this apparently simple situation.

We could choose to represent a volume with a standard `Float64`, but, as we shall see, there are good reasons to wrap this into a new type:

In [1]:
type Vol
    value::Float64
end

function f(x)
    x
end


f (generic function with 1 method)

[The keyword `type` is likely to change in a future version of Julia.] 

This defines a "template" for making objects (which are basically boxes that contain information), and specifies their internal structure; the resulting structure / shape is given the name `Vol`. Each is specified to contain a single object, a variable with the name `value` of type `Float64`. Note, however, that no actual object has yet been created.

To create an object of type `Vol`, we write

In [2]:
V = Vol(3)

Vol(3.0)

Note that this **constructor** (a function with the same name as the type that creates objects of that type) was **automatically generated** when we defined the type. We can ask Julia which of these so-called default constructors are provided:

In [3]:
methods(Vol)

# 3 methods for generic function "(::Type)":
Vol(value::Float64) at In[1]:2
Vol(value) at In[1]:2
(::Type{T}){T}(arg) at sysimg.jl:53

In [4]:
W = Vol(10.5)

Vol(10.5)

Trying to use 

In [5]:
Vol("David")

LoadError: LoadError: MethodError: `convert` has no method matching convert(::Type{Float64}, ::ASCIIString)
This may have arisen from a call to the constructor Float64(...),
since type constructors fall back to convert methods.
Closest candidates are:
  call{T}(::Type{T}, ::Any)
  convert(::Type{Float64}, !Matched::Int8)
  convert(::Type{Float64}, !Matched::Int16)
  ...
while loading In[5], in expression starting on line 1

gives an error, due to 

In [8]:
convert(Float64, "David")

LoadError: LoadError: MethodError: `convert` has no method matching convert(::Type{Float64}, ::ASCIIString)
This may have arisen from a call to the constructor Float64(...),
since type constructors fall back to convert methods.
Closest candidates are:
  call{T}(::Type{T}, ::Any)
  convert(::Type{Float64}, !Matched::Int8)
  convert(::Type{Float64}, !Matched::Int16)
  ...
while loading In[8], in expression starting on line 1

To interactively introspect and find out whath is inside the object `V`, we write `V.<TAB>`[press the `TAB` key after typing `V.` at the REPL, in the notebook, or in Atom].

Alternatively, we can use `fieldnames`, which returns an array of `Symbol`s, containing the names of the fields:

In [4]:
fieldnames(V)

1-element Array{Symbol,1}:
 :value

We access the field with any of the following commands:

In [15]:
V.value

3.0

In [16]:
getfield(V, :value)  # by name

3.0

In [17]:
getfield(V, 1)       # by number

3.0

In a `type`, we can also change the values of the variables in the object

In [5]:
V.value = 10

10

In [6]:
V

Vol(10.0)

We may want to prevent a user from altering the value stored in the object. To do this, use `immutable` instead of `type`:

In [7]:
immutable Vol2
    value::Float64
end

In [8]:
V2 = Vol2(3.5)

Vol2(3.5)

In [9]:
V2.value

3.5

In [10]:
V2.value = 10

LoadError: LoadError: type Vol2 is immutable
while loading In[10], in expression starting on line 1

## Constructors

So far, we have been using the default constructors. These have the following problem:

In [11]:
Vol(-1)

Vol(-1.0)

This should not be allowed, since volumes cannot be negative. We want to impose a constraint *at the moment when we  create the object*, i.e. in the constructor. To do so, we define an **inner constructor**, i.e. a constructor that is defined *inside* the type definition.

Since Julia does not allow redefining types, we need to clear the workspace before we do so:

In [12]:
workspace()

In [13]:
immutable Vol
    value::Float64
    
    function Vol(x)
        isa(x, Real) || throw(ArgumentError("Volume must be real."))
        x < 0 && throw(ArgumentError("Negative volume not allowed."))
        
        new(x)
    end
end

In [14]:
Vol("hola")

LoadError: LoadError: ArgumentError: Must be real
while loading In[14], in expression starting on line 1

In [15]:
Vol(-3)

LoadError: LoadError: ArgumentError: Negative volume not allowed.
while loading In[15], in expression starting on line 1

In [16]:
Vol(1)

Vol(1.0)

Here we have used so-called ["short-circuit" evaluation](http://docs.julialang.org/en/release-0.4/manual/control-flow/#man-short-circuit-evaluation): `&&` can be thought of as an `if...then`, while `||` is `unless...` (i.e. `if not ... then`).

The function `new` is a special function that instantiates (creates) the object. Using `new`, some of the fields may be left empty, by not including them in the call to `new`, and filled in later (if the type is not `immutable`), for example:

In [63]:
workspace()

type Example
    a::Float64
    b::Vector{Int}
    
    Example(a) = new(a)
end

Which constructors are now available?

In [66]:
methods(Example)

# 2 methods for generic function "(::Type)":
Example(a) at In[63]:7
(::Type{T}){T}(arg) at sysimg.jl:53

We see that defining our own inner constructor prevented Julia from automatically defining any constructors.

In [32]:
x = Example(3)

Example(3.0,#undef)

Here, `x.b` was not initialized. Accessing it produces an error:

In [33]:
x.b

LoadError: LoadError: UndefRefError: access to undefined reference
while loading In[33], in expression starting on line 1

But we can modify its value:

In [36]:
x.b = [1, 2]

x

Example(3.0,[1,2])

Note that the constructor that accepts both arguments is not defined:

In [65]:
Example(1, [1, 2])

LoadError: LoadError: MethodError: no method matching Example(::Int64, ::Array{Int64,1})
Closest candidates are:
  Example(::Any) at In[63]:7
  Example{T}(::Any) at sysimg.jl:53
while loading In[65], in expression starting on line 1

If we wish to define it, then we need to add it as another inner constructor:

In [67]:
workspace()

type Example
    a::Float64
    b::Vector{Int}
    
    Example(a) = new(a)
    Example(a, b) = new(a, b)
end

In [68]:
Example(1, [1, 2])

Example(1.0,[1,2])

## Making types parametric

There is no reason why volumes should be `Float64` -- they may be any kind of number. We could just remove the type annotation:

In [17]:
workspace()

In [18]:
immutable Vol
    value   # DO NOT DO THIS!
end

In [19]:
Vol(3.5)

Vol(3.5)

In [20]:
Vol(3)

Vol(3)

In [21]:
Vol("hello")

Vol("hello")

but we should **never do this**.  [By "never", we of course mean "only if you are really sure that you know what you're doing" and won't complain about the consequences.] The reason for this is that Julia is unable to infer the type of the variable, and so will be **very slow**.

Instead, what we really mean is that there should be a *different type* of `Vol` for each type that it could contain - integers, floats, `BigFloat`s, rationals, intervals, etc.
To tell Julia this, we use a **type parameter**, written with curly braces:

In [22]:
workspace()

In [37]:
immutable Vol{T}
    value::T   # DO THIS INSTEAD!
end

This says "for **any** type `T`, define a `Vol` with type parameter `T` as follows". This makes `Vol` a **parametric type**, a kind of meta-template (a template for templates). This turns out to be a surprisingly powerful concept when combined with multiple dispatch.  

The curly braces here, `{T}`, can be read as "for a given `T`, define `Vol`, parameterized with that `T`, as follows"; note that `T` is used in the body of the type definition. 

Again, default constructors are automatically provided:

In [38]:
Vol{Int64}(3)

Vol{Int64}(3)

In [39]:
Vol{Float64}(3.5)

Vol{Float64}(3.5)

The parameter `T` in the type definition means that when Julia sees *any* constructor called as `Vol{T}` for a given type `T`, it uses the body of the definition with `T` substituted with the corresponding type.

However, we are doing extra work that should be the job of the compiler: the compiler should be able to **infer** that `3` is an `Int64`, and indeed it can:

In [41]:
Vol(3)

Vol{Int64}(3)

In [42]:
Vol(3.5)

Vol{Float64}(3.5)

In [43]:
Vol("hello")

Vol{String}("hello")

In [44]:
?<:

search: <:



```
<:(T1, T2)
```

Subtype operator, equivalent to `issubtype(T1,T2)`.


Here we find a new problem: the volume should be a real number. We can specify this restriction using the **subtype operator**, `<:`:

In [45]:
workspace()

In [49]:
Int <: Real   # "is a subtype"

true

In [46]:
String <: Real

false

In [47]:
immutable Vol{T <: Real}  # only accept types T that are subtypes of Real
    value::T   
end

In [48]:
Vol("hello")

LoadError: LoadError: MethodError: Cannot `convert` an object of type String to an object of type Vol{T<:Real}
This may have arisen from a call to the constructor Vol{T<:Real}(...),
since type constructors fall back to convert methods.
while loading In[48], in expression starting on line 1

In [49]:
Vol(3)

Vol{Int64}(3)

Let's consider a more complicated example, representing a particle in one dimension. The particle has a position and a velocity, and we would like to restrict them to have the same, real type:

In [50]:
type Particle{T <: Real}
    x::T
    v::T
end

In [51]:
p = Particle(3, 4)

Particle{Int64}(3,4)

In [53]:
p2 = Particle(3, 4.5)

LoadError: LoadError: MethodError: no method matching Particle{T<:Real}(::Int64, ::Float64)
Closest candidates are:
  Particle{T<:Real}{T<:Real}(::T<:Real, !Matched::T<:Real) at In[50]:2
  Particle{T<:Real}{T}(::Any) at sysimg.jl:53
while loading In[53], in expression starting on line 1

We see that Julia is doing something quite sophisticated: it tries to fill in the types `T` *and* checks that types with the same parameter `T` do actually match. If they do, then it generates the corresponding type with the corresponding `T`.

What if we wanted to allow `Particle(3, 4.5)`, such that both got promoted to a common type? We do this by adding a new constructor:

In [54]:
Particle(a::Real, b::Real) = Particle(promote(a, b)...)

Particle{T<:Real}

In [55]:
Particle(3, 4.5)

Particle{Float64}(3.0,4.5)

In [56]:
promote(3, 4.5)

(3.0,4.5)

Here, the `...` operator is called a **splat**. It takes the tuple that is the result of `promote` and "undoes it", passing the values inside the tuple as individual arguments to the function `Particle`.
Without the splat, `Particle(promote(a, b))` would pass the tuple that is the result of `promote(a, b)` to the constructor `Particle` as a *single*, tuple argument.

**Exercise**: write a constructor of the `Particle` type that accepts a tuple.

## Inner constructors for parametric types

What happens if we try to add an inner constructor to a parametric type?

In [37]:
workspace()

In [38]:
immutable Vol{T<:Real}
    value::T   
    
    function Vol(x::Real)
        x < 0 && throw(ArgumentError("Negative volume not allowed."))
        
        new(x)
    end
end

In [39]:
Vol(3)

LoadError: LoadError: MethodError: Cannot `convert` an object of type Int64 to an object of type Vol{T<:Real}
This may have arisen from a call to the constructor Vol{T<:Real}(...),
since type constructors fall back to convert methods.
while loading In[39], in expression starting on line 1

It seems like we can no longer construct an object of this type. In fact we can, by explicitly specifying the type:

In [7]:
Vol{Int}(3)

Vol{Int64}(3)

Again, this is a pain and undesirable -- Julia should be able to infer the type for us.
The solution is to analyse what we want Julia to do, and then tell Julia to do exactly that. 

We would like Julia to look at the type of the object `x` passed to the constructor - call it `T` - and automatically use the constructor *for `Vol{T}`*, passing `x` as an argument, as we just did in the explicit call.

The syntax to do this looks strange at first (and second) glance:

In [40]:
Vol{T}(x::T) = Vol{T}(x)

Vol{T<:Real}

Now it works:

In [41]:
Vol(3)

Vol{Int64}(3)

In [42]:
methods(Vol)

# 2 methods for generic function "(::Type)":
(::Type{Vol}){T}(x::T) at In[40]:1
(::Type{T}){T}(arg) at sysimg.jl:53

What is going on here? We seem to almost be repeating ourselves, writing the same both on the left and right of the assignment operator `=`. Once again, we are telling Julia to treat `T` as a type parameter. However, the meaning of `{T}` is *different* on the two sides.

- The left-hand side means: "for **any** type `T`, define a function `Vol` that takes a parameter `x` *of type `T`*".
This can be thought of as a specification for a matching algorithm: look at the function call and see if it matches this template.


- The assignment operator has its usual meaning: "define the function on the left by the expression on the right".


- The right-hand side means: "call the constructor `Vol{T}` with argument `x`".

The point is that by the time the right-hand side is evaluated, Julia has **already realised** what `T` must be, since the function definition is matched by a call to `Vol` with an argument `x` of *some* type `T`, which Julia infers (i.e., it chooses `T` to match the type of the object that was passed in)!

The parametrised definition can be thought of as defining a "potentially infinite" number of different functions. One is instantiated, however, *only* if and when the function is actually called with an argument of that type.

Furthermore, Julia is sophisticated enough that we can make multiple definitions like this. For example, we may want to store all volumes that are integers as exact integers, but all floating-point volumes as `BigFloat`s. We can do this by making a special method for floats:

In [44]:
Vol{T<:AbstractFloat}(x::T) = Vol{BigFloat}(x)

Vol{T<:Real}

In [45]:
Vol(3.5)

Vol{BigFloat}(3.500000000000000000000000000000000000000000000000000000000000000000000000000000)

Passing in integers does *not* match this new definition, and so still uses the previous definition:

In [46]:
Vol(3)

Vol{Int64}(3)

In [47]:
methods(Vol)

# 3 methods for generic function "(::Type)":
(::Type{Vol}){T<:AbstractFloat}(x::T) at In[44]:1
(::Type{Vol}){T}(x::T) at In[40]:1
(::Type{T}){T}(arg) at sysimg.jl:53

This is *multiple dispatch** in action: when the generic function `Vol` is called, the correct method is called based on matching the type of the argument `T`.

## Arithmetic: functions on parametric types

We would like to be able to do arithmetic on objects of our type: 

In [48]:
V1 = Vol(3)
V2 = Vol(4)

Vol{Int64}(4)

In [49]:
V1 + V2

LoadError: LoadError: MethodError: no method matching +(::Vol{Int64}, ::Vol{Int64})
Closest candidates are:
  +(::Any, ::Any, !Matched::Any, !Matched::Any...) at operators.jl:138
while loading In[49], in expression starting on line 1

Since we have not specified what `+` means for objects of our type, Julia does not know what to do, so we must explicitly define `+` for objects of our type.

We may, for some reason, want to define the sum of volumes only if they contain objects of the same type. We can follow the idea and syntax discussed for the constructor:

In [51]:
import Base.+

+{T}(V1::Vol{T}, V2::Vol{T}) = Vol{T}(V1.value + V2.value)

+ (generic function with 164 methods)

This looks a bit noisy, but we just have to parse it piece by piece: it is a method of `+`, that is parametric with one type parameter `T`, and whose arguments are `Vol`s parametrised by **the same** `T`. This is important, since it means that the definition **will not match** function calls with two different types. This is again an aspect of multiple dispatch. (Strictly speaking, the previous example was single dispatch, since a single argument was involved. Here it is multiple dispatch, since there are two arguments to the function.)

Again, we can think of the `{T}` on the left as "define a template function that works for any `T`, as follows", and the `{T}` on the left is to refer to the parametric type `Vol{T}` with the `T` that has already been inferred (assigned) from the arguments `V1` and `V2`:

In [52]:
V1 + V2

Vol{Int64}(7)

Here, Julia examined `V1` and realised that it matches the definition when `T` is `Int64`.

However, this may be too restrictive; for example, we may wish to be able to add volumes with different integer types inside:

In [53]:
Vol{Int64}(1) + Vol{BigInt}(3)

LoadError: LoadError: MethodError: no method matching +(::Vol{Int64}, ::Vol{BigInt})
Closest candidates are:
  +(::Any, ::Any, !Matched::Any, !Matched::Any...) at operators.jl:138
  +{T}(::Vol{T}, !Matched::Vol{T}) at In[51]:3
while loading In[53], in expression starting on line 1

We can again define a more general method:

In [54]:
+{T1<:Integer, T2<:Integer}(V1::Vol{T1}, V2::Vol{T2}) = Vol{promote_type(T1,T2)}(V1.value + V2.value)

+ (generic function with 165 methods)

[We could have left out the type parameter on the right-hand side, but this leads to a method ambiguity warning, in which two different methods match the combination of argument types, in this case when we have two equal integer types.]

Here, we have *two* type parameters that are both subtypes of (`<:` again) the abstract `Integer` type, and `promote_type(T,S)` is a function from `Base` that determines the smallest supertype of both `T` and `S` that is able to correctly represent values of both types. 

In [55]:
Vol{Int64}(1) + Vol{BigInt}(3)

Vol{BigInt}(4)

In [56]:
Vol{Float64}(1) + Vol{BigFloat}(3)

LoadError: LoadError: MethodError: no method matching +(::Vol{Float64}, ::Vol{BigFloat})
Closest candidates are:
  +(::Any, ::Any, !Matched::Any, !Matched::Any...) at operators.jl:138
  +{T}(::Vol{T}, !Matched::Vol{T}) at In[51]:3
while loading In[56], in expression starting on line 1

Note that sometimes such a redefinition will apparently not work, since the previous definition, *even if it threw an error*, will be cached by Julia. In this case, it will be necessary to clear the workspace first and redefine everything. This can soon become tiresome and may be partially solved by moving the definitions out into a file, and `include`ing the file after calling `workspace()` to define the types. This kind of workflow discussion is best decided in practice. [In a future version of Julia, this caching of previous definitions will no longer apply; redefining a method will redefine all methods that depend on it.]

## Adding more parameters 

Types may be parametrized by more than one parameter. In the case of volumes, we should not be able to "add" a 1-dimensional volume (length) to a 2-dimensional one (area), since this makes no sense physically or mathematically; on the other hand, we can multiply them to get a 3-dimensional volume. We thus need to add an integer parameter to the `Vol` type:

In [68]:
workspace()

In [69]:
immutable Vol{N, T<:Real}
    value::T   
end

Again, a default constructor is created, but now we must explicitly specify the types, since the type `N` cannot be inferred from the argument:

In [70]:
Vol{3, Float64}(3)

Vol{3,Float64}(3.0)

represents a three-dimensional volume of magnitude 3.

We wish to define the sum *only* when the dimensions are the same:

In [71]:
import Base.+

+{N,T}(V1::Vol{N,T}, V2::Vol{N,T}) = Vol{N,T}(V1.value + V2.value)

+ (generic function with 166 methods)

This will match only when both arguments have the same parameters `T` and `N`. We could also promote the types:

In [72]:
+{N,T1,T2}(V1::Vol{N,T1}, V2::Vol{N,T2}) = Vol{N,promote_type(T1,T2)}(V1.value + V2.value)

+ (generic function with 167 methods)

We make a useful outer constructor that can infer the type of its argument, but still needs an explicit dimension:

In [73]:
Vol{T}(N, x::T) = Vol{N,T}(x)

Vol{N,T<:Real}

In [74]:
Vol(3, 4.5)

Vol{3,Float64}(4.5)

We would also like to display objects in a nicer way:

In [75]:
import Base.show
show{N,T}(io::IO, x::Vol{N,T}) = print(io, N, "-dimensional volume with magnitude ", x.value)

show (generic function with 213 methods)

In [76]:
A = Vol(2, 3.0)   # volume of dimension 2 with value 3.0

2-dimensional volume with magnitude 3.0

In [77]:
ℓ = Vol(1, 10.0)  # write as \ell<TAB>

1-dimensional volume with magnitude 10.0

In [78]:
A + ℓ

LoadError: LoadError: MethodError: no method matching +(::Vol{2,Float64}, ::Vol{1,Float64})
Closest candidates are:
  +(::Any, ::Any, !Matched::Any, !Matched::Any...) at operators.jl:138
  +{N,T}(::Vol{N,T}, !Matched::Vol{N,T}) at In[71]:3
  +{N,T1,T2}(::Vol{N,T1}, !Matched::Vol{N,T2}) at In[72]:1
while loading In[78], in expression starting on line 1

It is not nice for the user to throw this slightly cryptic error message. We can provide a better error message when volumes of different dimension are added; we should check that volumes of the same dimension *still works*. Such things should be codified in unit test suites.

In [79]:
workspace()

In [80]:
immutable Vol{N,T<:Real}
    value::T   
end

Vol{T}(N, x::T) = Vol{N,T}(x)

import Base.+

+{N,T}(V1::Vol{N,T}, V2::Vol{N,T}) = Vol{N,T}(V1.value + V2.value)
+{N1,N2,T}(V1::Vol{N1,T}, V2::Vol{N2,T}) = throw(ArgumentError("Volumes of different dimension cannot be added"))

+ (generic function with 169 methods)

In [81]:
V1 = Vol(3, 3)
V2 = Vol(3, 4)
V1 + V2

Vol{3,Int64}(7)

In [82]:
Vol(3, 3) + Vol(2, 4)

LoadError: LoadError: ArgumentError: Volumes of different dimension cannot be added
while loading In[82], in expression starting on line 1

**Exercise**
Define the product (`*`) of any two objects of type `Vol`.

## Conversion

We may want to define functions like `sin` that act on objects of our new type.
[We may *not* want to define such functions, on the other hand, since we could argue that a `Vol` is not a real number, but rather is a combination of a real number together with a dimension.]
Let's try it:

In [109]:
V1

Vol{3,Int64}(3)

In [110]:
sin(V1)

LoadError: LoadError: MethodError: `sin` has no method matching sin(::Vol{3,Int64})
while loading In[110], in expression starting on line 1

Of course, Julia doesn't know how to do this, since we haven't defined it.
However, an object of type `Vol` is just a simple wrapper around a number, so we could just say

In [83]:
import Base.sin
sin(x::Vol) = sin(x.value)

sin (generic function with 11 methods)

In [84]:
sin(V1)

0.1411200080598672

It is painful, boring and error-prone to do this for every function (although this may be done using metaprogramming). An alternative is to tell Julia explicitly that a `Vol` is actually a kind of real number, i.e. a subtype of `Real`.

Note that although this is not mathematically correct, it is a convenience to simplify the code.

[Future versions of Julia are slated to include **traits** (also called **interfaces**) to allow this behavior without pretending that types are subsets of other types.

In [85]:
workspace()

In [86]:
immutable Vol{N,T<:Real} <: Real
    value::T   
end

In [87]:
V = Vol{3,Float64}(3)

Vol{3,Float64}(3.0)

In [88]:
sin(V)

LoadError: LoadError: MethodError: Cannot `convert` an object of type Vol{3,Float64} to an object of type AbstractFloat
This may have arisen from a call to the constructor AbstractFloat(...),
since type constructors fall back to convert methods.
while loading In[88], in expression starting on line 1

We now get a different error. We can see which method Julia is now using:

In [89]:
@which sin(V)

sin(x::Real) at math.jl:204

This is a generic "fallback" ("catch-all") method, which is called when no more specialised method has been defined.

Following the link to the Julia source code, which is written in Julia and therefore amazingly understandable (with some work), and if we understand some metaprogramming, we find that the definition of `sin(x::Real)` is

    sin(x::Real) = sin(float(x))

We could just define `float` for our type, which is supposed to convert an object to it's most natural floating-point equivalent:

In [90]:
workspace()

In [91]:
immutable Vol{N,T<:Real} <: Real
    value::T   
end

import Base.float
float(V::Vol) = float(V.value)

float (generic function with 17 methods)

In [92]:
V = Vol{3,Float64}(3)
sin(V)

0.1411200080598672

However, in fact `float` is already defined for *any* `x::Real`:

In [93]:
workspace()

In [94]:
immutable Vol{N,T<:Real} <: Real
    value::T   
end

In [95]:
V = Vol{3,Float64}(3)

Vol{3,Float64}(3.0)

In [96]:
@which float(V)

float(x) at float.jl:152

In [97]:
float(3)

3.0

The definition reads:

    float(x) = convert(AbstractFloat, x)

Finally, we see that rather we need to investigate Julia's capabilities for [conversion and promotion](http://docs.julialang.org/en/release-0.4/manual/conversion-and-promotion/) and define how to `convert` an object of our type into a float:

In [98]:
import Base.convert
convert(::Type{AbstractFloat}, x::Vol) = float(x.value)

convert (generic function with 600 methods)

In [99]:
convert(AbstractFloat, V)

3.0

Now any standard unary function works:

In [101]:
sin(V), cos(V), exp(V)

(0.1411200080598672,-0.9899924966004454,20.085536923187668)

`::Type{AbstractFloat}` refers to an argument which has type "type of `AbstractFloat`"; the only object that has that type is `AbstractFloat` itself; this thus matches function calls of the form 

    convert(AbstractFloat, V)

## Promotion 

In the case of `Vol`, it is not natural to define all arithmetic operations, so let's simplify even further and just wrap a `Float64` in a `Prob` type, representing a probability. [We will assume that it makes sense to define all arithmetic operations here, although in a given application that may not be the case.]

In [ ]:
workspace()

In [102]:
immutable Prob{T<:AbstractFloat} <: Real
    _::T  # dummy name
    
    function Prob(x)
        (!(0 <= x <= 1)) && throw(ArgumentError("Probability must be between 0 and 1."))
        
        new(x)
    end
end

Prob{T<:AbstractFloat}(x::T) = Prob{T}(x)

Prob{T<:AbstractFloat}

[Note that `_` is a valid variable name in Julia, and is often used for a variable whose name we don't care about.]

In [103]:
x = Prob(0.5)

Prob{Float64}(0.5)

We see that multiplying `x` by a real number does not give a `MethodError`:

In [104]:
3x

LoadError: LoadError: no promotion exists for Int64 and Prob{Float64}
while loading In[104], in expression starting on line 1

since a `Vol` is a subtype of `Number`. 

Rather, Julia is trying to use its promotion machinery, in which such operations are handled by **promoting** both arguments to the same type:

In [105]:
@which 3x

*(x::Number, y::Number) at promotion.jl:191

We can see the process using `@which` (or using the debugger in Julia v0.5):

In [106]:
@which promote(3, x)

promote{T,S}(x::T, y::S) at promotion.jl:153

In [107]:
@which promote_type(Int, Prob{Float64})

promote_type{T,S}(::Type{T}, ::Type{S}) at promotion.jl:139

which is defined in terms of `promote_rule(T,S)`, e.g.

In [108]:
@which promote_rule(Float64, Int)

promote_rule(::Type{Float64}, ::Type{Int64}) at float.jl:56

So if we want to have promotion work, we need to define `promote_rule`:

In [109]:
workspace()

In [110]:
immutable Prob{T<:AbstractFloat} <: Real
    value::T
end

Prob{T<:AbstractFloat}(x::T) = Prob{T}(x)

import Base.*
*{T<:AbstractFloat}(x::Prob{T}, y::Prob{T}) = Prob{T}(x.value*y.value)

* (generic function with 150 methods)

In [111]:
p1 = Prob(0.3)
p2 = Prob(0.4)
p1 * p2

Prob{Float64}(0.12)

In [112]:
0.3 * p2

LoadError: LoadError: no promotion exists for Float64 and Prob{Float64}
while loading In[112], in expression starting on line 1

In [113]:
import Base: promote_rule, convert

promote_rule{S<:Real, T<:Real}(::Type{Prob{T}}, ::Type{S}) = Prob{promote_type(S, T)}

convert{S<:Real, T<:Real}(::Type{Prob{T}}, x::S) = Prob(float(x))
convert{T<:Real}(::Type{AbstractFloat}, x::Prob{T}) = x.value


convert (generic function with 602 methods)

In [114]:
x = Prob(0.5)


Prob{Float64}(0.5)

In [115]:
3x

Prob{Float64}(1.5)

In [116]:
promote(3, x)

(Prob{Float64}(3.0),Prob{Float64}(0.5))

These rules are always tricky to get right!

## `Vector`s of objects of parametric type

Suppose we make a vector of rationals, which is a parametric type

In [117]:
v = [3//4, 4//5]

2-element Array{Rational{Int64},1}:
 3//4
 4//5

In [118]:
typeof(v)

Array{Rational{Int64},1}

A **typealias**, that is, an alternative (shorter and/or more intuitive) name for the type `Array{T,1}` is `Vector{T}`:

In [119]:
Array{Rational{Int64},1} === Vector{Rational{Int}}  

true

Here, the `===` operator checks for identity (rather than just equality) of objects:

In [120]:
1 == 1.0

true

In [121]:
1 === 1.0

false

We also have a subtype relationship:

In [122]:
Rational{Int} <: Rational

true

where `Rational` is an abstract type, meaning that we cannot create objects of type `Rational`; they must always be parametrised.

**However**, we have

In [123]:
Vector{Rational{Int}} <: Vector{Rational}

false

Thus if we define a method

In [124]:
h(x::Vector{Rational}) = 2x

h (generic function with 1 method)

that accepts arguments that are `Vector`s of `Rational`s, we have

In [158]:
v = [3//4, 4//5]
h(v)

LoadError: LoadError: MethodError: `h` has no method matching h(::Array{Rational{Int64},1})
while loading In[158], in expression starting on line 2

i.e. `h` **does not** accept arguments of type `Vector{Rational{Int}}`. Instead, we must parametrize explicitly:

In [125]:
h{T}(x::Vector{Rational{T}}) = 2x

h (generic function with 2 methods)

In [126]:
h(v)

2-element Array{Rational{Int64},1}:
 3//2
 8//5

Here, the syntax `{T}` means "define a function `h` with this template, for *any* type `T`.

## Function wrappers (functors) and performance

Starting in Julia v0.5, functions are treated differently: the type of each function is unique:

In [131]:
typeof(sin)

Base.#sin

In [132]:
typeof(cos)

Base.#cos

In [133]:
typeof(x -> x)

##1#2

In [134]:
typeof(x -> x)

##3#4

These types are all subtypes of the abstract type `Function`:

In [138]:
supertype(typeof(sin)), supertype(typeof(x->x))

(Function,Function)

In [139]:
typeof(sin) <: Function

true

This allows us significant performance gains when functions are passed as parameters to other functions, such as `map`. (These latter functions are then called ["higher-order functions"](https://en.wikipedia.org/wiki/Higher-order_function).) It also allows performance gains when functions are wrapped in user-defined types that are supposed to behave like functions (["function objects"](https://en.wikipedia.org/wiki/Function_object), sometimes called "functors"). 

We might want to write the following to wrap a function and a value to call in a single object:

In [146]:
workspace()

In [147]:
type Wrapper1
    f::Function
    x::Float64
end

In [148]:
w = Wrapper1(sin, 10)

Wrapper1(sin,10.0)

We want to think of `w` itself as a function, but currently it cannot be called:

In [149]:
w()

LoadError: LoadError: MethodError: objects of type Wrapper1 are not callable
while loading In[149], in expression starting on line 1

We can define this as follows:

In [150]:
(w::Wrapper1)() = w.f(w.x)

In [153]:
w(), sin(10)

(-0.5440211108893698,-0.5440211108893698)

However, this is not performant. The main reason is that we could now do the following:

In [154]:
w.f = cos

cos (generic function with 10 methods)

In [156]:
w(), cos(10)

(-0.8390715290764524,-0.8390715290764524)

It is only *at run time*, when the function call is executed, that Julia knows which function is stored inside the object, and at that moment it must do a (slow) look-up.

To solve this problem, we parameterize the wrapper type:

In [158]:
type Wrapper2{F}
    f::F
    x::Float64
end

In [160]:
w = Wrapper2(sin, 10.0)

Wrapper2{Base.#sin}(sin,10.0)

Again, Julia infers the value of `F`, and creates an object of this new type, with a parameter `F` that is the type of `sin`. We again define how to call an object of this type, now with a slightly more complicated syntax, due to the parameter:

In [161]:
(w::Wrapper2{F}){F}() = w.f(w.x)

In [162]:
w()

-0.5440211108893698

The difference in performance can be measured. The dynamic look-up can be seen using `@code_llvm` or `@code_native`.

For an interesting discussion about how this is actually used to create very efficient complicated array structures, see [Tim Holy's keynote at JuliaCon 2016](https://www.youtube.com/watch?v=fl0g9tHeghA).

One caveat (trade-off) is that this approach requires a new version of `Wrapper2` to be compiled for each function that is passed to it.

## Abstract types and inheritance

A type of "inheritance" is possible in Julia. As a simple example, consider two types of objects in a simulation, particles, discs, and squares. These are all subtypes of objects that will have certain properties in common, for example that they can move. Each type of object has a position, but discs and squares also have other properties. 

We make an abstract type `MovableObject` and subtypes of it: 

In [164]:
abstract MovableObject

immutable Particle
    pos::Int
end

immutable Disc
    pos::Int
    radius::Float64
end

immutable Square
    pos::Int
    side::Float64
end

We could imagine that most objects will move by changing their position:

In [129]:
move(obj::MovableObject) = x += 1

move (generic function with 1 method)

but that `Square`s expand and contract by changing their side length

In [130]:
move(s::Square) = side += 1

move (generic function with 2 methods)

The `move` function applies to any `MovableObject`, but the specialised version of the function for `Square`s automatically takes priority.